In [1]:
import matplotlib
matplotlib.use('TkAgg')
import ecg_noise_detector.src.ecg_noise_detector.noiseDetector as nd

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC

In [36]:
import importlib
importlib.reload(nd)


<module 'ecg_noise_detector.src.ecg_noise_detector.noiseDetector' from 'C:\\Users\\daniel\\Documents\\CambridgeSoftwareProjects\\ecg-signal-quality\\ecg_noise_detector\\src\\ecg_noise_detector\\noiseDetector.py'>

In [2]:
dataset = pd.read_pickle("CinC2017Data/database.pk")

In [3]:
dataset["length"] = dataset["data"].map(lambda arr: arr.shape[-1])
# select only the 30s length records
dataset = dataset[dataset["length"] == 9000]
dataset["data"] = dataset["data"].map(lambda d: d[0])

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

In [26]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [5]:
# dataset["onehot"] = dataset["class"].map(generate_onehot)
def generate_index(c):
    if c == "N":
        return 0
    if c == "O":
        return 0
    if c == "A":
        return 0
    if c == "~":
        return 1

dataset["class_index"] = dataset["class"].map(generate_index)

train_dataset, test_dataset = train_test_split(dataset, test_size=0.15, stratify=dataset["class_index"])

print(len(test_dataset.index))

897


In [29]:
c = 0

num_rows = 4
num_cols = 4

num_class_samples = num_cols * num_rows
fig = make_subplots(rows=num_rows, cols=num_cols)

for i, (_, sample) in enumerate(dataset[dataset["class_index"] == c].sample(num_class_samples).iterrows()):
    fig.add_trace(go.Scatter(y=sample["data"]), row=i%num_cols + 1, col = i//num_rows + 1)

fig.update_layout(height=1000)
fig.update_xaxes(title="sample number")
fig.update_yaxes(title="amplitude")
fig.show()

In [6]:
print(dataset["class"].value_counts())

N    3695
O    1655
A     504
~     123
Name: class, dtype: int64


In [18]:
print(len(nd.get_example_ecg()))

15200


In [19]:
lt_yoav = nd._length_transfrom(test_dataset["data"][0], 100)

In [20]:
lt_mine = nd._length_transform_faster(test_dataset["data"][0], 100)

In [53]:
def test_code():
    ecgs = test_dataset["data"].head(20)
    nd.is_noisy_batch(ecgs, fs=300)

import cProfile
cProfile.run('test_code()')

C:\Users\daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVC from version 1.0.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\daniel\Documents\CambridgeSoftwareProjects\ecg-signal-quality\ecg_noise_detector\src\ecg_noise_detector\noiseDetector.py:81: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  p = pd.Series(index=ecgs.index)


Processing ecg: A04315
Processing ecg: A03779
Processing ecg: A05397
Processing ecg: A03823
Processing ecg: A00227
Processing ecg: A08170
Processing ecg: A03236
Processing ecg: A07489
Processing ecg: A04197
Processing ecg: A00860
Processing ecg: A04877
Processing ecg: A05200
Processing ecg: A01613
Processing ecg: A02253
Processing ecg: A05897
Processing ecg: A05041
Processing ecg: A07776
Processing ecg: A06002
Processing ecg: A07440
Processing ecg: A01792
         5192772 function calls (5113124 primitive calls) in 4.064 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.064    4.064 3875118959.py:1(test_code)
     2840    0.001    0.000    0.012    0.000 <__array_function__ internals>:177(all)
     3360    0.004    0.000    0.021    0.000 <__array_function__ internals>:177(amax)
     8480    0.015    0.000    0.058    0.000 <__array_function__ internals>:177(any)
     1720    0.001    0.000   

In [7]:
ecgs = test_dataset["data"]
test_dataset["predictions"] = nd.is_noisy_batch(ecgs, fs=300, filter=False)

C:\Users\daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVC from version 1.0.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\daniel\Documents\CambridgeSoftwareProjects\ecg-signal-quality\ecg_noise_detector\src\ecg_noise_detector\noiseDetector.py:81: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  p = pd.Series(index=ecgs.index)


Processing ecg: A00503
Processing ecg: A06949
Processing ecg: A05398
Processing ecg: A08103
Processing ecg: A05609
Processing ecg: A02815
Processing ecg: A07811
Processing ecg: A06506
Processing ecg: A06564
Processing ecg: A08068
Processing ecg: A08203
Processing ecg: A06871
Processing ecg: A00812
Processing ecg: A03623
Processing ecg: A00256
Processing ecg: A03910
Processing ecg: A03932
Processing ecg: A07280
Processing ecg: A07013
Processing ecg: A00022
Processing ecg: A02472
Processing ecg: A08055
Processing ecg: A07819
Processing ecg: A01137
Processing ecg: A07482
Processing ecg: A00415
Processing ecg: A04376
Processing ecg: A04510
Processing ecg: A03410
Processing ecg: A03156
Processing ecg: A07249
Processing ecg: A05010
Processing ecg: A08245
Processing ecg: A08237
Processing ecg: A03766
Processing ecg: A00189
Processing ecg: A00281
Processing ecg: A01817
Processing ecg: A00863
Processing ecg: A00908
Processing ecg: A03745
Processing ecg: A04434
Processing ecg: A03262
Processing 

In [8]:
print(test_dataset.head(5))

       class                                               data  length  \
A00503     O  [481, 632, 777, 904, 997, 1018, 1003, 906, 778...    9000   
A06949     N  [115, 138, 154, 165, 173, 181, 184, 179, 165, ...    9000   
A05398     O  [221, 252, 280, 307, 330, 349, 341, 317, 308, ...    9000   
A08103     N  [-906, -1261, -1705, -2261, -2914, -3620, -429...    9000   
A05609     O  [38, 39, 41, 44, 47, 51, 60, 82, 106, 127, 161...    9000   

        class_index predictions  
A00503            0       False  
A06949            0        True  
A05398            0        True  
A08103            0       False  
A05609            0        True  


In [9]:
test_dataset["predictions"] = test_dataset["predictions"].astype(int)

In [29]:
i = 100

print(test_dataset.iloc[i]["class"])
nd.plot_ecg(test_dataset.iloc[i]["data"], fs=300)

O


C:\Users\daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVC from version 1.0.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


<Figure size 1169x827 with 3 Axes>

In [10]:
conf_mat = confusion_matrix(test_dataset["class_index"], test_dataset["predictions"])

def F1_ind(conf_mat, ind):
    return (2 * conf_mat[ind, ind])/(np.sum(conf_mat[ind]) + np.sum(conf_mat[:, ind]))

print(f"Normal F1: {F1_ind(conf_mat, 0)}")
print(f"Other F1: {F1_ind(conf_mat, 1)}")

ConfusionMatrixDisplay.from_predictions(test_dataset["class_index"], test_dataset["predictions"], display_labels=["sufficient quality", "insufficient quality"], cmap="inferno")
plt.show()

Normal F1: 0.556920556920557
Other F1: 0.055846422338568937


In [11]:
print(f"Sensitivity: {conf_mat[1, 1]/np.sum(conf_mat[1])}")
print(f"Specificity: {conf_mat[0, 0]/np.sum(conf_mat[0])}")

Sensitivity: 0.8888888888888888
Specificity: 0.38680318543799774


In [32]:
false_positives = test_dataset[(test_dataset["predictions"] == 1) & (test_dataset["class_index"] == 0)]
false_negatives = test_dataset[(test_dataset["predictions"] == 0) & (test_dataset["class_index"] == 1)]
noisy = test_dataset[(test_dataset["predictions"] == 1) & (test_dataset["class_index"] == 1)]

In [25]:
nd.plot_ecg(false_positives.iloc[50]["data"], fs=300)

C:\Users\daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVC from version 1.0.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


<Figure size 1169x827 with 3 Axes>

In [30]:
nd.plot_ecg(false_negatives.iloc[0]["data"], fs=300)

C:\Users\daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning:

Trying to unpickle estimator SVC from version 1.0.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations



<Figure size 1169x827 with 3 Axes>

In [33]:
nd.plot_ecg(noisy.iloc[0]["data"], fs=300)

C:\Users\daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning:

Trying to unpickle estimator SVC from version 1.0.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations



<Figure size 1169x827 with 3 Axes>

## Try Training the SVM on the physionet data ourselves and see what the results are

In [37]:
ecg_features = []
i = 0

for ind, x in dataset.iterrows():
    print(f"Processing ecg {i}/{len(dataset.index)}\r", end="")
    df = nd._process_ecg(x["data"], fs=300, filter=False)
    df["class_index"] = x["class_index"]
    ecg_features.append(df)
    i += 1


df = pd.concat(ecg_features, keys=ecg_features[0].index)
new_ind = [(dataset.index[i1], i2) for i1, i2 in df.index] # idk how to apply this
df.index = pd.MultiIndex.from_tuples(new_ind)

In [38]:
df = pd.concat(ecg_features, keys=dataset.index)
print(df)

              sSQI      kSQI      pSQI    basSQI      bSQI      rSQI  \
A00001 0 -0.035735 -0.293814  0.473548  0.860109  0.666667  0.600000   
       1  0.640073 -0.233111  0.438357  0.881651  0.750000  0.571429   
       2  1.434288  0.470728  0.457808  0.973036  0.600000  0.625000   
       3  1.342493  0.340279  0.483638  1.026441  0.750000  0.571429   
       4  1.384905  0.373222  0.442891  1.009925  0.857143  0.538462   
...            ...       ...       ...       ...       ...       ...   
A08528 1  0.404828 -0.229772  0.212354  0.895093  0.416667  0.705882   
       2  0.817274  0.111169  0.231756  0.938942  0.500000  0.666667   
       3  0.469614 -0.176669  0.259132  0.943769  0.363636  0.733333   
       4  0.053038 -0.483399  0.205328  0.840294  0.416667  0.705882   
       5  0.346806 -0.309540  0.238891  0.888839  0.454545  0.687500   

          class_index  
A00001 0            0  
       1            0  
       2            0  
       3            0  
       4       

In [39]:
pk_path = "CinC2017Data/database_yeov_processed.pk"
df.to_pickle(pk_path)

In [40]:
print(ecg_features[0])

        sSQI      kSQI      pSQI    basSQI      bSQI      rSQI  class_index
0  -0.035735 -0.293814  0.473548  0.860109  0.666667  0.600000            0
1   0.025660 -0.183028  0.437098  0.851583  0.666667  0.600000            0
2   0.123664 -0.188683  0.420040  0.847567  0.666667  0.600000            0
3   0.153531 -0.032273  0.445362  0.925915  0.750000  0.571429            0
4   1.354541  0.397230  0.424141  0.971348  0.750000  0.571429            0
5   1.014704  0.190602  0.428096  0.947246  0.714286  0.538462            0
6   0.638997 -0.189993  0.453734  0.918943  0.857143  0.538462            0
7   0.605307 -0.222675  0.456757  0.900946  0.750000  0.571429            0
8   0.618544 -0.212560  0.462688  0.911559  0.714286  0.583333            0
9   0.650819 -0.193680  0.440739  0.904691  0.857143  0.538462            0
10  0.640073 -0.233111  0.438357  0.881651  0.750000  0.571429            0
11  0.642246 -0.203432  0.461666  0.916404  0.714286  0.583333            0
12  0.620645

In [12]:
df = pd.read_pickle("CinC2017Data/database_yeov_processed.pk")

In [14]:
df = df.iloc[:5000]

In [40]:
# Normalise some features to have 0 mean and variance 1 (other elements are already limited/normalised)

var_1_features = ["sSQI", "kSQI"]
for f in var_1_features:
    df[f] = (df[f] - df[f].mean())/df[f].var()

In [41]:
df.index.get_level_values(0)

Index(['A00001', 'A00001', 'A00001', 'A00001', 'A00001', 'A00001', 'A00002',
       'A00002', 'A00002', 'A00002',
       ...
       'A08527', 'A08527', 'A08527', 'A08527', 'A08528', 'A08528', 'A08528',
       'A08528', 'A08528', 'A08528'],
      dtype='object', length=35862)

In [42]:
ind = np.unique(np.array(df.index.get_level_values(0)))
classes = np.array([df["class_index"].loc[i, 0] for i in ind])

train_ind, test_ind = train_test_split(ind, test_size=0.15, stratify=classes)
train_dataset = df.loc[train_ind]
test_dataset = df.loc[test_ind]

print(train_dataset.head())
print(test_dataset.head())

              sSQI      kSQI      pSQI    basSQI      bSQI      rSQI  \
A07621 0 -0.618372 -1.669444  0.468557  0.958189  0.454545  0.687500   
       1  0.367216 -0.184676  0.358150  0.879411  0.454545  0.687500   
       2  1.120974  1.737762  0.287490  0.965157  0.625000  0.615385   
       3  0.344108 -0.469783  0.281874  0.812673  0.400000  0.666667   
       4  0.396270 -0.748469  0.316434  0.834349  0.555556  0.642857   

          class_index  
A07621 0            0  
       1            0  
       2            0  
       3            0  
       4            0  
              sSQI      kSQI      pSQI    basSQI      bSQI      rSQI  \
A06872 0 -0.719768 -2.203322  0.690891  0.548945  0.500000  0.666667   
       1 -0.066508 -0.912070  0.472648  0.671284  0.466667  0.681818   
       2 -0.302759 -2.173788  0.332832  0.746377  0.428571  0.700000   
       3 -0.805107 -2.240824  0.281326  0.676940  0.428571  0.700000   
       4 -0.856338 -1.979427  0.674716  0.593794  0.416667  0.6

In [43]:
# train the model

class_weights = 1/train_dataset["class_index"].value_counts()
class_weights /= np.sqrt(np.dot(class_weights, class_weights))
print(class_weights.to_dict())

model = SVC(class_weight=class_weights.to_dict())
# See documentation for default values e.g. use rbf, regularising C = 1
features = ["sSQI", "kSQI", "pSQI", "basSQI", "bSQI", "rSQI"]

model = model.fit(train_dataset[features].values, train_dataset["class_index"])

{0: 0.021100828550108818, 1: 0.9997773527313464}


In [44]:
print(test_dataset[features].head().values)

[[-0.71976769 -2.2033221   0.69089106  0.54894495  0.5         0.66666667]
 [-0.06650836 -0.91206984  0.4726475   0.67128405  0.46666667  0.68181818]
 [-0.30275929 -2.17378797  0.33283156  0.74637698  0.42857143  0.7       ]
 [-0.80510658 -2.24082413  0.28132647  0.67693987  0.42857143  0.7       ]
 [-0.85633794 -1.97942703  0.67471561  0.59379396  0.41666667  0.66666667]]


In [46]:
predictions = model.predict(test_dataset[features].values)
# THis is interesting I dropped the recombination of the sections!

test_dataset["predictions"] = predictions
# test_dataset["group_predicitons"] =
print(test_dataset.head())

ConfusionMatrixDisplay.from_predictions(test_dataset["class_index"], predictions, display_labels=["sufficint quality", "insufficient quality"], cmap="inferno")
plt.show()
conf_mat = confusion_matrix(test_dataset["class_index"], predictions)

def F1_ind(conf_mat, ind):
    return (2 * conf_mat[ind, ind])/(np.sum(conf_mat[ind]) + np.sum(conf_mat[:, ind]))

print(f"Normal F1: {F1_ind(conf_mat, 0)}")
print(f"Other F1: {F1_ind(conf_mat, 1)}")

              sSQI      kSQI      pSQI    basSQI      bSQI      rSQI  \
A06872 0 -0.719768 -2.203322  0.690891  0.548945  0.500000  0.666667   
       1 -0.066508 -0.912070  0.472648  0.671284  0.466667  0.681818   
       2 -0.302759 -2.173788  0.332832  0.746377  0.428571  0.700000   
       3 -0.805107 -2.240824  0.281326  0.676940  0.428571  0.700000   
       4 -0.856338 -1.979427  0.674716  0.593794  0.416667  0.666667   

          class_index  predictions  
A06872 0            0            1  
       1            0            0  
       2            0            1  
       3            0            1  
       4            0            1  
Normal F1: 0.8289342904472667
Other F1: 0.09362200117027501


In [47]:
print(f"Sensitivity: {conf_mat[1, 1]/np.sum(conf_mat[1])}")
print(f"Specificity: {conf_mat[0, 0]/np.sum(conf_mat[0])}")

Sensitivity: 0.7407407407407407
Specificity: 0.71160409556314
